In [2]:
from utils import * 
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
%matplotlib inline
import pickle
import random
from math import radians, sin, cos




In [3]:
# Contruction des fichiers de tests et images
if os.path.exists(MODELS_TEST_PATH):
    delete_files_in_directory(MODELS_TEST_PATH)
else:
    os.makedirs(MODELS_TEST_PATH)


def initFilesNames():
    filesNames = []
    for x in range(100, 1000, 100):
        filesNames.append("B0" + str(x + 1) + "T")
        filesNames.append("B0" + str(x + 2) + "T")
    filesNames.append("B0803T")
    filesNames.append("B0903T")
    return filesNames


DATA_FILES = initFilesNames()

ranges = [
    {"min": -0.5, "max": 3.5},
]
report = []
epocks = []
idx = 0

for r in ranges:
    for d in DATA_FILES:
        raw = file_preprocessing(
            d,
            lFilter=FILTERS[0]["l_cut"],
            hFilter=FILTERS[0]["h_cut"],
        )
        e = create_epochs(
            raw,
            tmin=r["min"],
            tmax=r["max"],
            reference=REFERENCE[0],
        )
        """plot_eeg_topomap_animation(
            e, "img" + str(idx)
        )"""
        epocks.append(e)
        idx += 1
XTrain, X_test, y_train, y_test = extract_features(epocks, True)
X_test.to_csv(MODELS_TEST_PATH + "Xtest.csv")
y_test.to_csv(MODELS_TEST_PATH + "ytest.csv")

"""for idx in range(0, 20):
    dfi = df.loc[df["id"] == idx].copy()
    dfi = dfi.drop(["id", "index"], axis=1)
    dfi["eventType"].to_csv(MODELS_TEST_PATH + "y" + str(idx) + ".csv")
    dfi = dfi.drop(["eventType"], axis=1)
    dfi.to_csv(MODELS_TEST_PATH + "t" + str(idx) + ".csv")"""

All files deleted successfully.
Creating RawArray with float64 data, n_channels=4, n_times=604803
    Range : 0 ... 604802 =      0.000 ...  2419.208 secs
Ready.
['STI101']
120 events found on stim channel STI101
Event IDs: [1 2]
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 825 samples (3.300 s)

['C3', 'Cz', 'C4', 'STI101']
120 events found on stim channel STI101
Event IDs: [1 2]
Not setting metadata
120 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using 

d:\perso\Informatique\Formations\ProjetData\mar24_cds_eeg\notebooks\hypothese2\utils.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfEvent.drop_duplicates(inplace=True)
d:\perso\Informatique\Formations\ProjetData\mar24_cds_eeg\notebooks\hypothese2\utils.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfEvent.drop_duplicates(inplace=True)


'for idx in range(0, 20):\n    dfi = df.loc[df["id"] == idx].copy()\n    dfi = dfi.drop(["id", "index"], axis=1)\n    dfi["eventType"].to_csv(MODELS_TEST_PATH + "y" + str(idx) + ".csv")\n    dfi = dfi.drop(["eventType"], axis=1)\n    dfi.to_csv(MODELS_TEST_PATH + "t" + str(idx) + ".csv")'

In [3]:
PATH_LENGTH = 10
NB_PREDICT = 5

def compute_point(point_to_be_rotated, angle, center_point = (0,0)):
    angle = radians(angle)
    xnew = cos(angle)*(point_to_be_rotated[0] - center_point[0]) - sin(angle)*(point_to_be_rotated[1] - center_point[1]) + center_point[0]
    ynew = sin(angle)*(point_to_be_rotated[0] - center_point[0]) + cos(angle)*(point_to_be_rotated[1] - center_point[1]) + center_point[1]
    return xnew,ynew

def prepare_pred(y_test):
        y_test = [-1 if num == 2 else num for num in y_test]
        y_test = np.insert(y_test,0,0)
        return y_test

def make_allways_turn_path(y_test):     
        lst = [0,1]
        y_test_ref = [0,0]
        for ind, y in  enumerate(y_test):
                if ind:
                        lst[ind], y_test_ref[ind] = compute_point((lst[ind], y_test_ref[ind]), 23 if y > 0  else -23, (lst[ind-1],y_test_ref[ind-1]))
                        if (ind < len(y_test) - 1):
                                lst.append(lst[ind] + (lst[ind] - lst[ind-1]))
                                y_test_ref.append(y_test_ref[ind] + (y_test_ref[ind] - y_test_ref[ind-1]))
        cubic_interpolation_model = interp1d(lst, y_test_ref, kind = "cubic")
        X_ = np.linspace(min(lst), max(lst), 83)
        return X_, cubic_interpolation_model(X_)
   
def make_turn_when_change_path(y_test):
        lst = range(0, len(y_test))
        y_test = [ sum(y_test[:ind+1]) if ind > 0 else x for ind, x in enumerate(y_test)]
        cubic_interpolation_model = interp1d(lst, y_test, kind = "cubic")       
        X_=np.linspace(min(lst), max(lst), 83)
        return X_, cubic_interpolation_model(X_)

def graph(method, y_test, y_pred_lg):
        X_, Y_ = method(y_test)
        plt.plot(X_, Y_, 'rv', linestyle='solid', label ='temoin')
        X_, Y_ = method(y_pred_lg)
        plt.plot(X_, Y_, 'b3', linewidth=0.5, linestyle='solid',label ='Lg')
        """X_, Y_ = method(y_pred_ada)
        plt.plot(X_, Y_, 'g1', linewidth=0.5, linestyle='solid', label ='Ada')"""
        plt.axis('off')
        plt.legend()
        plt.show()
        
X_test = pd.read_csv(MODELS_TEST_PATH + "X_test.csv")
y_test = pd.read_csv(MODELS_TEST_PATH + "y_test.csv")

lg_model = pickle.load(open(MODELS_PATH + "LogisticRegression", 'rb'))
#ada_model = pickle.load(open(MODELS_PATH + "AdaBoost", 'rb'))

samples = random.sample(range(0, y_test.shape[0]), PATH_LENGTH)

y_test = y_test.iloc[samples]['eventType']
X_test = X_test.iloc[samples].drop(['Unnamed: 0', 'id', 'index'], axis=1)

y_preds = [ lg_model.predict(X_test) for x in range(0, NB_PREDICT)]

#y_pred_ada = ada_model.predict(X_test)
y_pred_lg = [int(num) for num in y_pred_lg]
#y_pred_ada = [int(num) for num in y_pred_ada]
y_pred_lg = prepare_pred(y_pred_lg)
#y_pred_ada = prepare_pred(y_pred_ada)
y_test =  prepare_pred(y_test)

graph(make_allways_turn_path, y_test, y_pred_lg)
graph(make_turn_when_change_path, y_test, y_pred_lg)




FileNotFoundError: [Errno 2] No such file or directory: '../../models/test/X_test.csv'